<a href="https://colab.research.google.com/github/Sung-Yao/DRL_HW5/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")


In [ ]:
query_environment("MountainCar-v0")

Action Space: Discrete(3)
Observation Space: Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)
Max Episode Steps: 200
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: -110.0


In [ ]:
query_environment("CartPole-v1")

Action Space: Discrete(2)
Observation Space: Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
Max Episode Steps: 500
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: 475.0


In [ ]:
query_environment("MountainCarContinuous-v0")

Action Space: Box(-1.0, 1.0, (1,), float32)
Observation Space: Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)
Max Episode Steps: 999
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: 90.0


In [ ]:
# HIDE OUTPUT
!wget http://www.atarimania.com/roms/Roms.rar 
!unrar x -o+ /content/Roms.rar >/dev/nul
!python -m atari_py.import_roms /content/ROMS >/dev/nul

--2022-06-17 12:05:07--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19583716 (19M) [application/x-rar-compressed]
Saving to: ‘Roms.rar’

Roms.rar            100%[===================>]  18.68M  4.64MB/s    in 4.1s    

2022-06-17 12:05:11 (4.54 MB/s) - ‘Roms.rar’ saved [19583716/19583716]



In [ ]:
query_environment("Breakout-v0")

Action Space: Discrete(4)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: 10000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


In [ ]:
query_environment("Breakout-ram-v0")

Action Space: Discrete(4)
Observation Space: Box(0, 255, (128,), uint8)
Max Episode Steps: 10000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# **安裝顯示Atari遊戲的library**

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install stable_baselines3 gym box2d-py tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 13.9 MB/s 
     |████████████████████████████████| 448 kB 71.4 MB/s 
     |████████████████████████████████| 511.7 MB 7.2 kB/s 
     |████████████████████████████████| 1.5 MB 42.9 MB/s 
     |████████████████████████████████| 438 kB 82.6 MB/s 
     |████████████████████████████████| 5.8 MB 45.7 MB/s 
     |████████████████████████████████| 1.6 MB 54.2 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616798 sha256=5323f84f7c6f11394f382d9f80616faec17ad8cf0310a51fdf455bba1a32e236
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
Successfully built gym
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstall

# **improt相關libaray，並讓測試及訓連完成的影片在colab上播放**

In [ ]:
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

#實現gym環境的影片錄製和顯示
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env


# **初步隨機測試**

In [ ]:
#env = wrap_env(gym.make("MountainCar-v0"))
env = wrap_env(gym.make("Atlantis-v0"))

observation = env.reset()
reward = 0 #初始化reward
while True:

    env.render()

    # your agent goes here
    action = env.action_space.sample()

    observation, rewards, done, info = env.step(action)
    reward += rewards
    if done:
        break
print(reward)
env.close()
show_video()


13600.0


# **訓練模型**

In [ ]:
env = wrap_env(gym.make("Atlantis-v0") )#選擇要訓練的gym
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1) #用PPO的model

Using cpu device
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=100000) #訓練次數設定

-----------------------------
| time/              |      |
|    fps             | 171  |
|    iterations      | 1    |
|    time_elapsed    | 11   |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 37         |
|    iterations           | 2          |
|    time_elapsed         | 108        |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01275944 |
|    clip_fraction        | 0.0791     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.38      |
|    explained_variance   | 3.77e-05   |
|    learning_rate        | 0.0003     |
|    loss                 | 1.01e+05   |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.00639   |
|    value_loss           | 1.64e+05   |
----------------------------------------
-----------------------------------------
| time/   

In [ ]:
env.close() #關閉訓練環境

In [ ]:
model.save("/content/drive/MyDrive/deep reinforcement learning/colab/fight_PPO_model_100000") #將model儲存到雲端硬碟的路徑

In [ ]:
model = PPO.load("/content/drive/MyDrive/deep reinforcement learning/colab/fight_PPO_model_100000", env=env) #更改讀取雲端硬碟model的路徑

Wrapping the env in a VecTransposeImage.


# **讀取model後再測試**

In [ ]:
env = wrap_env(gym.make("Atlantis-v0"))
obs = env.reset() #obs重置
done = False
reward = 0 #初始化reward
while not done:
    action, _states = model.predict(obs.copy())
    obs, rewards, done, info = env.step(action)
    reward+=rewards
    env.render()
print(reward)

env.close()
show_video()


40000.0
